# EOEPCA OGC API Processes (OAPIP) Validation and Usage Notebook

## Setup

In [ ]:
import os
import requests
import json
from pathlib import Path

import sys
sys.path.append('../')
from modules.helpers import get_access_token, load_eoepca_state, test_cell, test_results

Load `eoepca state` environment

In [ ]:
load_eoepca_state()

In [ ]:
platform_domain = os.environ.get("INGRESS_HOST")
use_oidc = os.environ.get("OIDC_OAPIP_ENABLED", "true") == "true"
oapip_user = os.environ.get("KEYCLOAK_TEST_USER", "eoepcauser")
oapip_domain = f'https://zoo.{platform_domain}'

# oidc only
oapip_client_id = os.environ.get("OAPIP_CLIENT_ID", "oapip")
oapip_client_secret = os.environ.get("OAPIP_CLIENT_SECRET")
username = os.environ.get("KEYCLOAK_TEST_USER")
password = os.environ.get("KEYCLOAK_TEST_PASSWORD")

## Authentication Setup (Conditional)

In [ ]:
if use_oidc:
    # Fetch OIDC Token via Keycloak
    from modules.helpers import get_access_token

    """
    ⚙️ !! You may need to re-run this cell to generate a new access token if it expires whilst using the notebook !! ⚙️
    """
    access_token = get_access_token(username, password, oapip_client_id, oapip_client_secret)
    headers = {"Authorization": f"Bearer {access_token}"}
    print("✅ OIDC Authentication setup complete.")
else:
    headers = {}
    print("⚠️ OIDC Authentication not enabled; proceeding without authentication.")


## Validate API Endpoints

In [ ]:
endpoints = [
    ("Swagger UI", f"{oapip_domain}/swagger-ui/oapip/"),
    ("OGC API Processes Landing", f"{oapip_domain}/{oapip_user}/ogc-api/")
]

for name, url in endpoints:
    response = requests.get(url, headers=headers)
    print(f"{name} ({url}): {response.status_code}")

## List Available Processes

In [ ]:
processes_url = f"{oapip_domain}/{oapip_user}/ogc-api/processes"
response = requests.get(processes_url, headers=headers)

if response.status_code == 200:
    processes = response.json()
    print(f"✅ Available processes:")
    print(json.dumps(processes, indent=2))
else:
    print(f"❌ Failed to retrieve processes. Status code: {response.status_code}")

## Deploy a Sample Process (`convert-url`)

In [ ]:
deploy_url = processes_url
deploy_payload = {
    "executionUnit": {
        "href": "https://raw.githubusercontent.com/EOEPCA/deployment-guide/refs/heads/main/scripts/processing/oapip/examples/convert-url-app.cwl",
        "type": "application/cwl"
    }
}

response = requests.post(deploy_url, headers={**headers, "Content-Type": "application/json"}, json=deploy_payload)

if response.status_code in [200, 201]:
    print(f"✅ Successfully deployed 'convert-url' process.")
    deployed_process = response.json()
    print(json.dumps(deployed_process, indent=2))
else:
    print(f"❌ Failed to deploy 'convert-url'. Status code: {response.status_code}, Response: {response.text}")

## Execute the `convert-url` Process

In [ ]:
execute_url = f"{processes_url}/convert-url/execution"
execute_payload = {
    "inputs": {
        "fn": "resize",
        "url": "https://eoepca.org/media_portal/images/logo6_med.original.png",
        "size": "50%"
    }
}

response = requests.post(
    execute_url,
    headers={
        **headers,
        "Content-Type": "application/json",
        "Prefer": "respond-async"
    },
    json=execute_payload
)

if response.status_code in [200, 201]:
    job_location = response.headers["Location"]
    job_id = response.json().get("jobID")
    print(f"✅ Execution started successfully. Job ID: {job_id}")
    print(f"Job monitoring URL: {job_location}")
else:
    print(f"❌ Failed to start execution. Status code: {response.status_code}, Response: {response.text}")

## Monitor Execution Status

In [ ]:
status_url = f"{oapip_domain}/{oapip_user}/ogc-api/jobs/{job_id}"
status = "running"
import time
print("Monitoring job execution...")
while status.lower() == "running":
    time.sleep(5)
    print("Checking job status...")
    response = requests.get(status_url, headers=headers)
    if response.status_code == 200:
        job_status = response.json()
        status = job_status.get("status")
        print(f"Status: {status}")
        if status.lower() == "running":
            time.sleep(5)  # wait before rechecking
    else:
        print(f"❌ Failed to get job status. Status code: {response.status_code}")
        break

print("Final job status:", status)


## Retrieve Execution Result

In [ ]:
results_url = f"{status_url}/results"
response = requests.get(results_url, headers=headers)

if response.status_code == 200:
    results = response.json()
    print(f"✅ Job results retrieved successfully:")
    print(json.dumps(results, indent=2))
else:
    print(f"❌ Failed to retrieve results. Status code: {response.status_code}, Response: {response.text}")

## Undeploy the `convert-url` Process

In [ ]:
undeploy_url = f"{processes_url}/convert-url"
response = requests.delete(undeploy_url, headers=headers)

if response.status_code in [200, 204]:
    print(f"✅ Successfully undeployed 'convert-url' process.")
else:
    print(f"❌ Failed to undeploy 'convert-url'. Status code: {response.status_code}, Response: {response.text}")